# SQL - Анализ базы данных книг

<a id="toc"></a>

## Оглавление

1. [Описание проекта](#describe)
   1. [Постановка задачи](#task)
2. [Обзор данных](#view)
3. [Описание данных](#data)
4. [Решение задач](#queries)
   1. [Сколько книг вышло после 01.01.2000](#q1)
   2. [Подсчет количества обзоров и средней оценки для каждой книги](#q2)
   3. [Самое большое издательство](#q3)
   4. [Автор с самой высокой средней оценкой книг](#q4)
   5. [Среднее количество обзоров от пользователей](#q5)
5. [Выводы](#resume)

<a id="describe"></a>

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

Наша первая задача как аналитика — проанализировать базу данных.<br>
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

<a id="task"></a>

### Постановка задачи

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

<a id="view"></a>

## Обзор данных

[К оглавлению](#toc)

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'
} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

def q_exec(query='SELECT version()', show=False):
    '''
    Функция принимает запрос на языке sql и возвращает объект типа pandas DataFrame
    ## Parameters:
    `query` - запрос в виде строки. по умолчанию "SELECT version()"
    `show` - вывести сразу результат на экран без присваивания результата работы функции переменной, по умолчанию `False`
    '''
    if show:
        return display(pd.io.sql.read_sql(query, con = engine))
    else:
        return pd.io.sql.read_sql(query, con = engine)

In [3]:
# function test
q_exec()

,version
0,PostgreSQL 11.17 (Ubuntu 11.17-201) on x86_64-...


Схема таблиц данных:

![hpeg](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2)

<a id="data"></a>

## Описание данных

[К оглавлению](#toc)

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Выведем первые строки каждой таблицы:

In [4]:
# напишем функцию для сбора информации о датафрейме

def df_info(df=pd.DataFrame()):
    '''
    Функция выводит на экран 5 случайных строк и 
    общую информацию о датафрейме
    ## Parameters:
    `df` - объект pandas.DataFrame
    '''
    display(df.sample(5))
    print('-----------')
    print(df.info(), 'Явных дубликатов - ', df.duplicated().sum())

In [5]:
# таблица books
books_query = 'SELECT * FROM books'

df_info(q_exec(books_query))

,book_id,author_id,title,num_pages,publication_date,publisher_id
635,636,561,The Art of War,273,2005-01-11,264
271,272,287,Furies of Calderon (Codex Alera #1),504,2005-06-28,1
347,348,426,In the Heart of the Sea: The Tragedy of the Wh...,302,2001-05-01,212
181,182,450,Dead Souls,464,2004-07-29,217
951,952,581,Utopia,135,2003-05-06,217


-----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None Явных дубликатов -  0


In [6]:
# таблица authors
authors_query = 'SELECT * FROM authors'

df_info(q_exec(authors_query))

,author_id,author
507,508,Robert Ludlum
227,228,Ian McEwan
395,396,Mark Kurlansky
87,88,Charles Frazier
568,569,Tennessee Williams


-----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None Явных дубликатов -  0


In [7]:
# таблица publishers
publishers_query = 'SELECT * FROM publishers'

df_info(q_exec(publishers_query))

,publisher_id,publisher
267,268,Simon Schuster
209,210,Penguin
30,31,Back Bay Books
13,14,Amistad
80,81,Dell Laurel-Leaf


-----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None Явных дубликатов -  0


In [8]:
# таблица ratings
ratings_query = 'SELECT * FROM ratings'

df_info(q_exec(ratings_query))

,rating_id,book_id,username,rating
2381,2382,379,abigailmoore,3
3190,3191,500,anthonyanderson,2
1000,1001,166,ewerner,5
2430,2431,392,yvonnevillarreal,2
6225,6226,953,jnelson,5


-----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None Явных дубликатов -  0


In [9]:
# таблица reviews
reviews_query = 'SELECT * FROM reviews'

df_info(q_exec(reviews_query))

,review_id,book_id,username,text
54,55,19,shannonsutton,Order land bank. Tonight somebody create sit o...
2405,2405,863,kevincampbell,Help mouth approach say what help stage. Movie...
176,176,62,joseph14,Program most of race tough. Far treatment baby...
2320,2320,834,richard89,Rest thus ever knowledge identify. Candidate l...
530,529,194,taylorjeffrey,Money pattern protect heavy. Bill event option...


-----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None Явных дубликатов -  0


**Выводы:**

* в нашем распоряжении 5 таблиц:
  * `books` - 1000 книг, типы данных соответствуют определнию (кроме `publication_date`),
  * `authors` - 636 авторов книг,
  * `publishers` - 340 издателей,
  * `ratings` - 6456 оценок пользователей,
  * `reviews` - 2793 пользовательских обзора.
* пропусков и явных дубликатов нет.

<a id="queries"></a>

## Решение задач

[К оглавлению](#toc)

<a id="q1"></a>

### Подсчет количества книг, вышедших после 1 января 2000 года

In [10]:
q1 = '''
        SELECT count(book_id) 
        FROM books WHERE publication_date > '01.01.2000';
    '''

answer_1 = q_exec(q1)

answer_1 = int(answer_1.values[0])

answer_1

819

<a id="q2"></a>

### Подсчет количества обзоров и средней оценки для каждой книги

[К оглавлению](#toc)

In [11]:
q2 = '''
        WITH
                reviews_qty
        AS

        (SELECT DISTINCT
                b.title,
                b.book_id,
                COUNT(rw.review_id) AS review_qty
                
        FROM books b
            LEFT JOIN reviews rw
            USING(book_id)
  
        GROUP BY b.title, b.book_id
        ORDER BY 2 desc)

        SELECT  
                rq.title,
                rq.review_qty,
                AVG(r.rating)
        FROM reviews_qty rq
        LEFT JOIN ratings r
        USING(book_id)
        GROUP BY rq.title, rq.review_qty
        ORDER BY 2 desc
'''

answer_2 = q_exec(q2)

answer_2.head()

,title,review_qty,avg
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,Eat Pray Love,6,3.395833
4,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645


**Вывод:**

* больше всего обзоров получила книга `Twilight (Twilight #1)` — `7` штук, а вот рейтинг у нее — `3.7/10`.

<a id = "q3"></a>

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

[К оглавлению](#toc)

In [12]:
q3 = '''
        SELECT
                p.publisher,
                COUNT(p.publisher_id) AS publisher_qty
                
        FROM books b
            LEFT JOIN publishers p
            USING(publisher_id)
            
        WHERE
                b.num_pages > 50
                
        GROUP BY p.publisher_id
        ORDER BY 2 DESC
        LIMIT 1;      
'''

answer_3 = q_exec(q3)

answer_3 = answer_3['publisher'].values[0]

answer_3

'Penguin Books'

<a id="q4"></a>

### Определить автора с самой высокой средней оценкой книг - учитываются только книги с 50 и более оценками

[К оглавлению](#toc)

In [13]:
q4 = '''
        SELECT
                a.author,
                AVG(rating) AS average_rating
                
        FROM books b
            LEFT JOIN authors a
            USING(author_id)
            LEFT JOIN ratings r
            USING(book_id)   
             
        GROUP BY a.author
        HAVING COUNT(book_id) > 50
        ORDER BY 2 DESC
        LIMIT 100
'''

answer_4 = q_exec(q4)

answer_4 = answer_4['author'].values[0]

answer_4

'J.K. Rowling/Mary GrandPré'

<a id="q5"></a>

### Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

[К оглавлению](#toc)

In [14]:
q5 = '''
        WITH 
        
        users_above_50_rates AS
        
        (SELECT
                r.username,
                COUNT(r.username)
        FROM
                ratings r
        LEFT JOIN books b
        USING(book_ID)
        GROUP BY r.username
        HAVING COUNT(r.username) > 50)

        SELECT 
                COUNT(username) / COUNT(DISTINCT username)
        
        FROM users_above_50_rates ur
                
        LEFT JOIN reviews rw
        USING(username)
             
'''

answer_5 = q_exec(q5)

answer_5 = int(answer_5.values[0])

answer_5

24

<a id="resume"></a>

## Выводы

[К оглавлению](#toc)

Мы загрузили данные каждой из таблиц, сделали SQL-запросы и получили ответы на 5 вопросов.

*Обзор данных:*
* в нашем распоряжении 5 таблиц:
  * `books` - 1000 книг, типы данных соответствуют определнию (кроме `publication_date`),
  * `authors` - 636 авторов книг,
  * `publishers` - 340 издателей,
  * `ratings` - 6456 оценок пользователей,
  * `reviews` - 2793 пользовательских обзора.
* пропусков и явных дубликатов нет.
  
*Результаты каждой задачи:*

* `819` книг вышло после 1 января 2000 года,
* количество обзоров и средней оценки для каждой книги хранится в фрейме `answer_2`, больше всего обзоров получила книга `Twilight (Twilight #1)` — `7` штук, а вот рейтинг у нее — `3.7/10`.
* `Penguin Books` - издательство, которое выпустило наибольшее число книг толще 50 страниц,
* `J.K. Rowling/Mary GrandPré` - автор с самой высокой средней оценкой книг (учитывались только книги с 50 и более оценками),
* `24` - среднее количество обзоров от пользователей, которые поставили больше 50 оценок.